# Update Oxford Nanopore tables

This notebook updates the 'sample' table of available ONT flowcells, 'sample_set' table (one or more flowcells per participant), and 'participant' table (list of unique individuals discovered).

To auto-populate these tables, this notebook scans files in the gs://broad-gp-oxfordnano and gs://broad-dsde-methods-long-reads-deepseq buckets and extracts relevant metadata from the final_summary.txt files.

If changes were made to the 'sample' table in Terra, we take care not to overwrite those changes. If one wishes to restore those entries to their original values, the rows should first be deleted from the Terra table.

All other tables are auto-generated based on the 'sample' table.

## Setup

Install and import some packages that we're going to need.  Set up some helper functions.

In [1]:
!pip install --use-feature=2020-resolver --upgrade pip pandas_gbq google-cloud-storage google-cloud-bigquery fastnumbers xmltodict > /dev/null 2>/dev/null

In [2]:
import os
import re
import hashlib

import pandas as pd
import firecloud.api as fapi

from google.cloud import bigquery
from google.cloud import storage
from google.api_core.exceptions import NotFound

from collections import OrderedDict

import xmltodict
import pprint

In [3]:
def load_summaries(gcs_buckets):
    storage_client = storage.Client()
    schemas = OrderedDict()

    ts = []
    for gcs_bucket in gcs_buckets:
        blobs = storage_client.list_blobs(re.sub("^gs://", "", gcs_bucket))

        for blob in blobs:
            if 'final_summary' in blob.name:
                doc = blob.download_as_string()
                t = {}
                
                for line in doc.decode("utf-8").split("\n"):
                    if '=' in line:
                        k,v = line.split('=')
                    
                        t[k] = v
                        
                t['Files'] = {
                    'final_summary.txt': gcs_bucket + "/" + blob.name,
                    'sequencing_summary.txt': 'missing'
                }

                bs = storage_client.list_blobs(re.sub("^gs://", "", gcs_bucket), prefix=os.path.dirname(blob.name) + "/" + t['sequencing_summary_file'])
                for b in bs:
                    t['Files']['sequencing_summary.txt'] = gcs_bucket + "/" + b.name
                    
                if 'sequencing_summary.txt' not in t['Files']:
                    pp = pprint.PrettyPrinter(indent=4)
                    pp.pprint(t)

                ts.append(t)

    return ts


def upload_sample_set(namespace, workspace, tbl):
    # delete old sample set
    ss_old = fapi.get_entities(namespace, workspace, f'sample_set').json()
    sample_sets = list(map(lambda e: e['name'], ss_old))
    f = [fapi.delete_sample_set(namespace, workspace, sample_set_index) for sample_set_index in sample_sets]

    # upload new sample set
    ss = tbl.filter(['participant'], axis=1).drop_duplicates()
    ss.columns = [f'entity:sample_set_id']
    
    b = fapi.upload_entities(namespace, workspace, entity_data=ss.to_csv(index=False, sep="\t"), model='flexible')
    if b.status_code == 200:
        print(f'Uploaded {len(ss)} sample sets successfully.')
    else:
        print(b.json())
    
    # upload membership set
    ms = tbl.filter(['participant', 'entity:sample_id'], axis=1).drop_duplicates()
    ms.columns = [f'membership:sample_set_id', f'sample']
    
    c = fapi.upload_entities(namespace, workspace, entity_data=ms.to_csv(index=False, sep="\t"), model='flexible')
    if c.status_code == 200:
        print(f'Uploaded {len(ms)} sample set members successfully.')
    else:
        print(c.json())


## Environment

Set up our environment (Terra namespace, workspace, and the location of ONT bucket(s)).

In [4]:
namespace = os.environ['GOOGLE_PROJECT']
workspace = os.environ['WORKSPACE_NAME']
default_bucket = os.environ['WORKSPACE_BUCKET']

gcs_buckets_ont = ['gs://broad-gp-oxfordnano', 'gs://broad-dsde-methods-long-reads-deepseq']

## Retrieve existing sample table from Terra

If it exists, retrieve the 'sample' table from this workspace.

In [5]:
ent_old = fapi.get_entities(namespace, workspace, 'sample').json()

if len(ent_old) > 0:
    tbl_old = pd.DataFrame(list(map(lambda e: e['attributes'], ent_old)))
    tbl_old["entity:sample_id"] = list(map(lambda f: hashlib.md5(f.encode("utf-8")).hexdigest(), tbl_old["final_summary_file"]))

## Examine Oxford Nanopore final_summary.txt files

Navigate GCS directories and look for ONT flowcells (indicated by the presence of the final_summary.txt files).

In [ ]:
ts = load_summaries(gcs_buckets_ont)

In [ ]:
tbl_header = ["final_summary_file", "sequencing_summary_file", "protocol_group_id", "instrument", "position", "flow_cell_id", "original_participant_name", "participant", "basecalling_enabled", "started", "acquisition_stopped", "processing_stopped", "fast5_files_in_fallback", "fast5_files_in_final_dest", "fastq_files_in_fallback", "fastq_files_in_final_dest"]
tbl_rows = []

for e in ts:
    tbl_rows.append([
        e["Files"]["final_summary.txt"],
        e["Files"]["sequencing_summary.txt"],
        e["protocol_group_id"],
        e["instrument"],
        e["position"],
        e["flow_cell_id"],
        e["sample_id"],
        e["sample_id"],
        e["basecalling_enabled"],
        e["started"],
        e["acquisition_stopped"],
        e["processing_stopped"],
        e["fast5_files_in_fallback"],
        e["fast5_files_in_final_dest"],
        e["fastq_files_in_fallback"],
        e["fastq_files_in_final_dest"]
    ])
    
tbl_new = pd.DataFrame(tbl_rows, columns=tbl_header)
tbl_new["entity:sample_id"] = list(map(lambda f: hashlib.md5(f.encode("utf-8")).hexdigest(), tbl_new["final_summary_file"]))

## Merge old and new sample list

If there are changes to the old sample list, make sure we retain them through subsequent table updates.  Do not overwrite old sample entries (metadata may have been manually modified).

In [ ]:
if len(ent_old) > 0:
    for sample_id in tbl_old.merge(tbl_new, how='outer', indicator=True).loc[lambda x : x['_merge']=='left_only']['entity:sample_id'].tolist():
        print(f'Entry for sample {sample_id} has been modified.  Keeping changes.')
        tbl_new = tbl_new[tbl_new['entity:sample_id'] != sample_id]

In [ ]:
merged_tbl = pd.merge(tbl_old, tbl_new, how='outer') if len(ent_old) > 0 else tbl_new
merged_tbl = merged_tbl[['entity:sample_id'] + tbl_header]

## Upload new sample table to Terra

Upload the merged 'sample' table to this workspace.

In [ ]:
a = fapi.upload_entities(namespace, workspace, entity_data=merged_tbl.to_csv(index=False, sep="\t"), model='flexible')

if a.status_code == 200:
    print(f'Uploaded {len(merged_tbl)} rows successfully.')
else:
    print(a.json())

## Upload sample_set table to Terra

Create a 'sample_set' table that groups flowcells by participant name.

In [ ]:
upload_sample_set(namespace, workspace, merged_tbl)